In [18]:
import os


In [2]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate

Found existing installation: transformers 4.46.3
Uninstalling transformers-4.46.3:
  Successfully uninstalled transformers-4.46.3
Found existing installation: accelerate 1.0.1
Uninstalling accelerate-1.0.1:
  Successfully uninstalled accelerate-1.0.1


In [3]:
pip install transformers accelerate

  Using cached transformers-4.46.3-py3-none-any.whl.metadata (44 kB)
  Using cached accelerate-1.0.1-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.46.3-py3-none-any.whl (10.0 MB)
Using cached accelerate-1.0.1-py3-none-any.whl (330 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pwd

'c:\\Projects\\Text-Summarizer\\research'

In [5]:
os.chdir('../')

In [6]:
%pwd

'c:\\Projects\\Text-Summarizer'

In [7]:
pip uninstall protobuf -y


Found existing installation: protobuf 3.20.3
Uninstalling protobuf-3.20.3:
  Successfully uninstalled protobuf-3.20.3
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install protobuf==3.20.3


  Using cached protobuf-3.20.3-cp38-cp38-win_amd64.whl.metadata (699 bytes)
Using cached protobuf-3.20.3-cp38-cp38-win_amd64.whl (904 kB)
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install sentencepiece==0.1.99


Note: you may need to restart the kernel to use updated packages.


In [10]:
from dataclasses import dataclass
from pathlib import Path

In [11]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [12]:
import sys

# Add 'src' folder to system path
sys.path.append(os.path.join(os.getcwd(), "src"))

In [13]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [15]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

c:\Users\asmit\miniconda3\envs\TextSummarization\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-07-08 16:45:12,092: INFO: config: PyTorch version 2.4.1 available.]


In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"


        import gc
        torch.cuda.empty_cache()
        gc.collect()
        

        model_ckpt = self.config.model_ckpt
        tokenizer = AutoTokenizer.from_pretrained(model_ckpt, use_fast=True)
        model_ = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_)
        
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
        #     weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
        #     evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
        #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # ) 


        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            eval_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        ) 

        train_dataset = dataset_samsum_pt["test"].select(range(100))
        eval_dataset = dataset_samsum_pt["validation"].select(range(20))

        trainer = Trainer(
            model=model_,
            args=trainer_args,
            tokenizer=tokenizer,
            data_collator=seq2seq_data_collator,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset
        )


        
        

        
        trainer.train()

        ## Save model
        model_.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [22]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    trainer = ModelTrainer(config=model_trainer_config)
    trainer.train()
except Exception as e:
    raise e

[2025-07-08 23:37:04,173: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-08 23:37:04,185: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-08 23:37:04,192: INFO: common: created directory at: artifacts]
[2025-07-08 23:37:04,198: INFO: common: created directory at: artifacts/model_trainer]


  0%|          | 0/6 [6:43:11<?, ?it/s]
c:\Users\asmit\miniconda3\envs\TextSummarization\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\asmit\AppData\Local\Temp\ipykernel_22508\1410292050.py:45: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  0%|          | 0/6 [00:00<?, ?it/s]

IndexError: index out of range in self